Here we'll try to find out the value in row 10 of our database(consisting of binary values) by doing a differencing attack.

In [1]:
# The way it is intended to work is by performing the same query twice on the dataset while including and excluding the value 
# at the targeted row(here, row 10). Then we find the difference between the two queries(hence the name). Thus, a non-zero 
# difference indicates a 1 and vice versa.

In [18]:
# We start with database generation functions

import torch

def get_parallel_db(db,remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        parallel_dbs.append(get_parallel_db(db, i))
    
    return parallel_dbs

def create_db_and_pdbs(num_entries):
    
    db = (torch.rand(num_entries) > 0.5).float()
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

In [19]:
db, _ = create_db_and_pdbs(100)

In [20]:
# we remove the value at row 10
pdb = get_parallel_db(db, remove_index=10)

In [21]:
# lets take a database with value 1 at row 10
db[10]

tensor(1.)

In [22]:
# differencing attack using sum() query

sum(db) - sum(pdb)

tensor(1.)

In [23]:
# As expected we get non-zero(1) output 
# Now we can also do a differencing attack using a mean query
sum(db).float()/len(db) - sum(pdb).float()/len(pdb)
# This will again result in a non-zero output

tensor(0.0059)

In [32]:
sum(pdb)
(sum(db).float() > 42).float()

tensor(0.)

In [33]:
# Differencing attack using a threshold query
(sum(db).float() > 41).float() - (sum(pdb).float() > 41).float()

tensor(1.)